In [1]:
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
# Loading .csv from github raw
url = 'https://raw.githubusercontent.com/umaimehm/Intro_to_AI/master/assignment2-machineLearning/NAS.csv'
nas = pd.read_csv(url)
nas.head(30)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2003-12-18,19.482599,19.596901,19.025499,19.139799,19.139799,4978496.0
1,2003-12-19,19.368299,19.425501,18.282801,18.454201,18.454201,1410901.0
2,2003-12-22,18.739901,18.739901,17.997101,18.054300,18.054300,137047.0
3,2003-12-23,17.997101,17.997101,17.368700,17.425800,17.425800,229418.0
4,2003-12-24,NaN,NaN,NaN,NaN,NaN,NaN
5,2003-12-25,NaN,NaN,NaN,NaN,NaN,NaN
6,2003-12-26,NaN,NaN,NaN,NaN,NaN,NaN
7,2003-12-29,17.425800,17.425800,16.854500,17.254400,17.254400,196206.0
8,2003-12-30,17.425800,17.882900,17.425800,17.711500,17.711500,67298.0
9,2003-12-31,NaN,NaN,NaN,NaN,NaN,NaN


## Feature engineering

There are alot of rows in the dataset which has no value. These are dates in which the stock market are closed, such as sundays, weekends and national holidays.


In [3]:
# Find the total of rows with no data value
nas.isnull().sum()

Date          0
Open         35
High         35
Low          35
Close        35
Adj Close    35
Volume       35
dtype: int64

In [6]:
# Copy old dataset and clean it
new_nas = nas.copy()
new_nas = new_nas.dropna(axis="rows")
new_nas.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [11]:
# Comparing rows in old and new dataset
print(f"The old dataset had a total of {nas.shape[0]} rows while the cleaned dataset has a total of {new_nas.shape[0]}. This means {nas.shape[0]-new_nas.shape[0]} rows has been removed.")

The old dataset had a total of 4253 rows while the cleaned dataset has a total of 4218. This means 35 rows has been removed.
